<a href="https://colab.research.google.com/github/Sanjanathakal/BIA-678/blob/master/ProjectBIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading data from the drive :

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing Required Libraries: 

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax


Importing training and testing csv : 

In [ ]:
user_data=pd.read_csv("/content/drive/MyDrive/BIA PROJECT /DATA/train_full.csv")
vendor_data = pd.read_csv("/content/drive/MyDrive/BIA PROJECT /DATA/vendors.csv")

user_data.head()
vendor_data.head()

In [4]:
user_data.head()

,customer_id,gender,status_x,verified_x,created_at_x,updated_at_x,location_number,location_type,latitude_x,longitude_x,id,authentication_id,latitude_y,longitude_y,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,commission,is_akeed_delivering,discount_percentage,status_y,verified_y,rank,language,vendor_rating,sunday_from_time1,sunday_to_time1,sunday_from_time2,sunday_to_time2,monday_from_time1,monday_to_time1,monday_from_time2,monday_to_time2,tuesday_from_time1,tuesday_to_time1,tuesday_from_time2,tuesday_to_time2,wednesday_from_time1,wednesday_to_time1,wednesday_from_time2,wednesday_to_time2,thursday_from_time1,thursday_to_time1,thursday_from_time2,thursday_to_time2,friday_from_time1,friday_to_time1,friday_from_time2,friday_to_time2,saturday_from_time1,saturday_to_time1,saturday_from_time2,saturday_to_time2,primary_tags,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at_y,updated_at_y,device_type,display_orders,location_number_obj,id_obj,CID X LOC_NUM X VENDOR,target
0,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,4,118597.0,-0.5884,0.7544,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,-,15,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,"{""primary_tags"":""4""}",1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1,0,4,TCHWPBT X 0 X 4,0
1,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,13,118608.0,-0.4717,0.7446,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,-,14,0.0,Yes,0.0,1.0,1,11,EN,4.7,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,"{""primary_tags"":""7""}",1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1,0,13,TCHWPBT X 0 X 13,0
2,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,20,118616.0,-0.4075,0.6436,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,-,19,0.0,Yes,0.0,1.0,1,1,EN,4.5,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,"{""primary_tags"":""71""}",1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1,0,20,TCHWPBT X 0 X 20,0
3,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,23,118619.0,-0.5854,0.7540,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,-,16,0.0,Yes,0.0,1.0,1,11,EN,4.5,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,"{""primary_tags"":""46""}",1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1,0,23,TCHWPBT X 0 X 23,0
4,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,28,118624.0,0.4807,0.5527,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,-,10,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,"{""primary_tags"":""32""}",1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1,0,28,TCHWPBT X 0 X 28,0


In [6]:
vendor_data.head()

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,commission,is_akeed_delivering,discount_percentage,status,verified,rank,language,vendor_rating,sunday_from_time1,sunday_to_time1,sunday_from_time2,sunday_to_time2,monday_from_time1,monday_to_time1,monday_from_time2,monday_to_time2,tuesday_from_time1,tuesday_to_time1,tuesday_from_time2,tuesday_to_time2,wednesday_from_time1,wednesday_to_time1,wednesday_from_time2,wednesday_to_time2,thursday_from_time1,thursday_to_time1,thursday_from_time2,thursday_to_time2,friday_from_time1,friday_to_time1,friday_from_time2,friday_to_time2,saturday_from_time1,saturday_to_time1,saturday_from_time2,saturday_to_time2,primary_tags,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,-,15,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,"{""primary_tags"":""4""}",1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,-,14,0.0,Yes,0.0,1.0,1,11,EN,4.7,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,"{""primary_tags"":""7""}",1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.407527,0.643681,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,-,19,0.0,Yes,0.0,1.0,1,1,EN,4.5,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,"{""primary_tags"":""71""}",1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.585385,0.753811,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,-,16,0.0,Yes,0.0,1.0,1,11,EN,4.5,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,"{""primary_tags"":""46""}",1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.480602,0.552850,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,-,10,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,"{""primary_tags"":""32""}",1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


In [9]:
vendor_data.dtypes

id                        int64
authentication_id       float64
latitude                float64
longitude               float64
vendor_category_en       object
vendor_category_id      float64
delivery_charge         float64
serving_distance        float64
is_open                 float64
OpeningTime              object
OpeningTime2             object
prepration_time           int64
commission              float64
is_akeed_delivering      object
discount_percentage     float64
status                  float64
verified                  int64
rank                      int64
language                 object
vendor_rating           float64
sunday_from_time1        object
sunday_to_time1          object
sunday_from_time2        object
sunday_to_time2          object
monday_from_time1        object
monday_to_time1          object
monday_from_time2        object
monday_to_time2          object
tuesday_from_time1       object
tuesday_to_time1         object
tuesday_from_time2       object
tuesday_

In [10]:
user_data.dtypes

customer_id               object
gender                    object
status_x                   int64
verified_x                 int64
created_at_x              object
                           ...  
display_orders             int64
location_number_obj        int64
id_obj                     int64
CID X LOC_NUM X VENDOR    object
target                     int64
Length: 73, dtype: object

In [11]:
user_data[['customer_id','id','vendor_rating']]

,customer_id,id,vendor_rating
0,TCHWPBT,4,4.4
1,TCHWPBT,13,4.7
2,TCHWPBT,20,4.5
3,TCHWPBT,23,4.5
4,TCHWPBT,28,4.4
...,...,...,...
5802395,LBMRK7A,849,4.1
5802396,LBMRK7A,855,4.2
5802397,LBMRK7A,856,4.3
5802398,LBMRK7A,858,4.2


In [12]:
vendor_data[['id','vendor_rating']]

,id,vendor_rating
0,4,4.4
1,13,4.7
2,20,4.5
3,23,4.5
4,28,4.4
...,...,...
95,849,4.1
96,855,4.2
97,856,4.3
98,858,4.2


In [14]:
resturants = vendor_data['id'].unique()
print(len(resturants))
print(resturants)

100
[  4  13  20  23  28  33  43  44  55  66  67  75  76  78  79  81  82  83
  84  85  86  90  92 104 105 106 110 113 115 134 145 148 149 154 157 159
 160 161 176 180 188 189 191 192 193 195 196 197 199 201 203 207 216 221
 225 231 237 243 250 259 265 271 274 288 289 294 295 298 299 300 303 304
 310 356 386 391 398 401 419 459 537 547 573 575 577 578 582 583 676 679
 681 841 843 845 846 849 855 856 858 907]


In [15]:
users = user_data['customer_id'].unique()
print(len(users))
print(users)

34523
['TCHWPBT' 'ZGFSYCZ' 'S2ALZFL' ... 'SQMJ08H' '9LW9CHN' 'LBMRK7A']


In [20]:
newdata = pd.DataFrame(user_data, columns=['customer_id','id','vendor_rating'])

In [24]:
print(newdata)

        customer_id   id  vendor_rating
0           TCHWPBT    4            4.4
1           TCHWPBT   13            4.7
2           TCHWPBT   20            4.5
3           TCHWPBT   23            4.5
4           TCHWPBT   28            4.4
...             ...  ...            ...
5802395     LBMRK7A  849            4.1
5802396     LBMRK7A  855            4.2
5802397     LBMRK7A  856            4.3
5802398     LBMRK7A  858            4.2
5802399     LBMRK7A  907            4.3

[5802400 rows x 3 columns]


In [25]:
newdata.to_csv('newdata.csv')
!cp newdata.csv "/content/drive/MyDrive/BIA PROJECT /DATA"

In [26]:
data = pd.DataFrame(columns=newdata.columns)

In [35]:
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm
def create_utility_matrix(data, formatizer = {'user':0, 'Resturant': 1, 'Rating': 2}):
    """
        :param data:      Array-like, 2D, nx3
        :param formatizer:pass the formatizer
        :return:          utility matrix (n x m), n=users, m=items
    """
        
    ResturantField = formatizer['Resturant']
    userField = formatizer['user']
    RatingField = formatizer['Rating']
    userList = data.iloc[:,userField].tolist()
    ResturantList = data.iloc[:,ResturantField].tolist()
    RatingList = data.iloc[:,RatingField].tolist()
    users = list(set(data.iloc[:,userField]))
    Resturants = list(set(data.iloc[:,ResturantField]))
    users_index = {users[i]: i for i in range(len(users))}
    pd_dict = {Resturant: [np.nan for i in range(len(users))] for Resturant in Resturants}
    for i in range(0,len(data)):
        Resturant = ResturantList[i]
        user = userList[i]
        Rating = RatingList[i]
    pd_dict[Resturant][users_index[user]] = Rating
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    Resturantcols = list(X.columns)
    Resturant_index = {Resturantcols[i]: i for i in range(len(Resturantcols))}
    # users_index gives us a mapping of user_id to index of user
    # Resturants_index provides the same for Resturants
    return X, users_index, Resturant_index

In [ ]:
create_utility_matrix(newdata)

In [40]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 34.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619420 sha256=c35cb9798ba8cf6315c90fcd44fe4bb80b96e39dc71dfe43e4f4016ca44561f7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [50]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import cross_validate

In [52]:
df = pd.DataFrame(newdata)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['customer_id','id','vendor_rating']], reader)

 

In [53]:
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0049  0.0049  0.0049  0.0049  0.0049  0.0049  0.0000  
MAE (testset)     0.0037  0.0037  0.0037  0.0037  0.0037  0.0037  0.0000  
Fit time          282.28  287.31  290.22  290.86  288.08  287.75  3.03    
Test time         26.30   26.38   25.18   23.18   29.08   26.02   1.91    


{'fit_time': (282.2827801704407,
  287.31149101257324,
  290.2210099697113,
  290.8615036010742,
  288.08062505722046),
 'test_mae': array([0.00370587, 0.00370348, 0.00371814, 0.00371565, 0.00371072]),
 'test_rmse': array([0.00488807, 0.00488582, 0.0049015 , 0.00490208, 0.00488852]),
 'test_time': (26.300243854522705,
  26.382837057113647,
  25.175880193710327,
  23.18099546432495,
  29.07761573791504)}

In [54]:
testdata = pd.read_csv("/content/drive/MyDrive/BIA PROJECT /DATA/testdata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,4,5,7,14,19,20,23,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,60,61,62,65,66) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
trainset = data.build_full_trainset()

In [58]:
algo.fit(trainset)
